In [1]:
import openai
import langchain
import pinecone
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain_community.vectorstores import Pinecone
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader #reads the document
from langchain.text_splitter import RecursiveCharacterTextSplitter #splits the text
from langchain.embeddings.openai import OpenAIEmbeddings #for the vector conversion
from langchain.vectorstores import Pinecone as LangchainPinecone
from langchain.llms import OpenAI
from dotenv import load_dotenv
load_dotenv()
import os
import pandas as pd
from langchain.docstore.document import Document


In [23]:
load_dotenv()
os.environ['PINECONE_API_KEY'] = '99cdfa3f-3924-44af-ad30-b86b04f33fa2'
os.environ['OPENAI_API_KEY'] = 'sk-proj-pftKuqx1XdcekOpurKB4T3BlbkFJMIJYhwJNHZeHDkxac8xO'

In [24]:
# reading the document
def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents

In [25]:
directory= r"C:\Users\96650\Downloads\GenAI.csv"
df = pd.read_csv(directory)
doc=read_doc(directory)
len(doc)
len(df)
#convert DataFrame to a list of dictionaries
documents = df.to_dict(orient='records')
len(documents)

97

In [26]:
for index, row in df.iterrows():
    print(f"Header: {row['Header']}")
    print(f"Paragraph: {row['Paragraph']}")
    print("-" * 50) 

Header: Information and services
Paragraph: The UAE Strategy for Government Services, Unified Digital Platform Policy and Digital Customer and Digital Government Service Policy are all a part of the UAE Government’s efforts to provide advanced digital services, accessible from anywhere and at any time.
--------------------------------------------------
Header: Service statistics
Paragraph: The UAE provides more than 6,000 federal and local digital services. At the federal level, there are 1,913 federal digital services. They can be classified as follows:
--------------------------------------------------
Header: 5,414,388 +
Paragraph: Issue a new work permit
--------------------------------------------------
Header: 4,879,775 +
Paragraph: Issue an entry permit
--------------------------------------------------
Header: 3,450,760 +
Paragraph: Electricity and water bill payment
--------------------------------------------------
Header: 2,509,670 +
Paragraph: Payment of traffic fines
-----

In [27]:
for doc in documents:
    print(doc)
    print("-" * 50)  #just a separater for better readability


{'Header': 'Information and services', 'Paragraph': 'The UAE Strategy for Government Services,\xa0Unified Digital Platform Policy\xa0and\xa0Digital Customer and Digital Government Service Policy\xa0are all a part of the UAE Government’s efforts to\xa0provide advanced digital services, accessible from anywhere and at any time.'}
--------------------------------------------------
{'Header': 'Service statistics', 'Paragraph': 'The UAE provides more than 6,000 federal and local digital services. At the federal level, there are 1,913\xa0federal digital services. They can be classified as follows:'}
--------------------------------------------------
{'Header': '5,414,388 +', 'Paragraph': 'Issue a new work permit'}
--------------------------------------------------
{'Header': '4,879,775 +', 'Paragraph': 'Issue an entry permit'}
--------------------------------------------------
{'Header': '3,450,760 +', 'Paragraph': 'Electricity and water bill payment'}
---------------------------------------

In [28]:

doc_objects = []
for doc in documents:
    page_content = f"{doc['Header']}\n{doc['Paragraph']}"  # Combine Header and Paragraph
    doc_objects.append(Document(page_content=page_content, metadata={}))

In [29]:
#divide the docs into chunks

def chunk_data(docs, chunk_size=500, chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return docs

In [30]:
documents = chunk_data(doc_objects)

In [38]:

index = pinecone.Index("langchainvector")
print(index.describe_index_stats())

TypeError: Index.__init__() missing 1 required positional argument: 'host'

In [37]:
#initialize embedding technique (openai)
embeddings= OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])
pc = Pinecone(index="langchain", embeddings=embeddings, text_key="text")#this is what is used to convert text into vectors

TypeError: Pinecone.__init__() got an unexpected keyword argument 'embeddings'

In [ ]:
vectors=embeddings.embed_query("How are you?")
len(vectors)

1536

In [ ]:
index_name="langchainvector"

vectorstore_from_texts = PineconeVectorStore.from_texts(
    chunk_data(docs=documents),
    index_name=index_name,
    embedding=embeddings
    )


In [ ]:
Pinecone.from_documents(documents,embeddings,index_name=index_name)

In [ ]:
#vector search DB in pinecone
pinecone.init(
    api_key=os.environ['PINECONE_API_KEY'],
    environment='us-east-1-aws'
)
index_name="langchainvector"

AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )



In [ ]:
pinecone.init(
    api_key="99cdfa3f-3924-44af-ad30-b86b04f33fa2",
    cloud="aws", 
    region="us-east-1"
)
index_name="langchainvector"